# Violin plot

In [9]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# from plot_utils import plotly_template
import plotly.express as px

# import data base
main_dir = '/home/mchevillard/disks/meso_shared/gaze_exp/'
pp_dir = "{}/derivatives/pp_data".format(main_dir)
tsv_dir = '{}/sub-all/prf/tsv'.format(pp_dir)

data = pd.read_table('{}/group_task-prf_prf.tsv'.format(tsv_dir))
data = data.drop(columns=['Unnamed: 0'])
data_loo = pd.read_table('{}/group_task-loo_prf_prf.tsv'.format(tsv_dir))
data_loo = data_loo.drop(columns=['Unnamed: 0'])

In [10]:
# define parameters
ecc_th = [0,15]
size_th= [0.1,20]
r2_th = 0.1

# Replace all data outer threshold with NaN data
data.loc[(data.ecc < ecc_th[0]) | (data.ecc > ecc_th[1]) | (data.sd < size_th[0]) | (data.sd > size_th[1]) | (data.r2 <=r2_th)] = np.nan
data = data.dropna()
data_loo.loc[(data_loo.ecc<ecc_th[0]) | (data_loo.ecc > ecc_th[1]) | (data_loo.sd < size_th[0]) | (data_loo.sd > size_th[1]) | (data_loo.r2 <=r2_th)] = np.nan
data_loo = data_loo.dropna()

In [11]:
rois = pd.unique(data.roi)
mask = pd.notnull(data.subject)
subjects = pd.unique(data.subject[mask])
rois_loo = pd.unique(data_loo.roi)
mask_loo = pd.notnull(data_loo.subject)
subjects_loo = pd.unique(data_loo.subject[mask_loo])

In [12]:
fig_height, fig_width = 1080,1920

fig_1_R2 = go.Figure()
fig_2_R2 = go.Figure()
fig_1_SD = go.Figure()
fig_2_SD = go.Figure()
fig_1_ecc = go.Figure()
fig_2_ecc = go.Figure()
fig_1_ecc_loo = go.Figure()
fig_2_ecc_loo = go.Figure()

In [13]:
# roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]
roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

roi_colors_loo = []
for color in roi_colors:
    updated_color = f"rgba{color[3:-1]}, {0.5})"
    roi_colors_loo.append(updated_color)

## Violin plot R2 - Visual areas for each subject

In [19]:
for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        fig=fig_1_R2 if i==0 else fig_2_R2
        
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.r2, name=roi, line_color = roi_colors[j], meanline_visible=True))

        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='R2')
        fig.update_xaxes(title_text='Visual areas')
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0.25, y=1, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

    fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
    fig.write_image('/home/mchevillard/disks/meso_S/{}_r2_roi.pdf'.format(subject))

## Violin plot Size - Visual areas for each subject

In [20]:
for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        fig=fig_1_SD if i==0 else fig_2_SD
        
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.sd, name=roi, line_color = roi_colors[j], meanline_visible=True))

        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='pRF size (dva)')
        fig.update_xaxes(title_text='Visual areas')
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0.25, y=20, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

    fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
    fig.write_image('/home/mchevillard/disks/meso_S/{}_size_roi.pdf'.format(subject))

## Violin plot Eccentricity - Visual areas for each subject

In [21]:
for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        fig=fig_1_ecc if i==0 else fig_2_ecc
        
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.ecc, name=roi, line_color = roi_colors[j], meanline_visible=True))

        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='pRF eccentricity (dva)')
        fig.update_xaxes(title_text='Visual areas')
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0.25, y=15, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

    fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
    fig.write_image('/home/mchevillard/disks/meso_S/{}_ecc_roi.pdf'.format(subject))

## Violin plot shared Eccentricity - Visual areas for each subject

In [12]:
for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        df_loo = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi == roi)]
        fig=fig_1_ecc_loo if i==0 else fig_2_ecc_loo
        
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.ecc, name=roi, line_color = roi_colors[j], points=False, side='negative', meanline_visible=True))
        fig.add_trace(go.Violin(x=df_loo['roi'][df_loo['roi']==roi], y=df_loo.ecc, name=roi, line_color = roi_colors_loo[j], points=False, side='positive', meanline_visible=True, fillcolor='rgb(255,255,255)'))

        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='pRF eccentricity (dva)')
        fig.update_xaxes(title_text='Visual areas')
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0, y=16, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

    fig.update_layout(violingap=0, violinmode='overlay', height=fig_height, width=fig_width, template='simple_white')
    fig.write_image('/home/mchevillard/disks/meso_S/{}_loo_ecc_roi.pdf'.format(subject))

## Violin plot pCM - Visual areas for each subject

In [21]:
# for i, subject in enumerate(subjects):
#     for j, roi in enumerate(rois):
#         df = data.loc[(data.subject == subject) & (data.roi == roi)]
#         fig=fig_1_ecc if i==0 else fig_2_ecc
        
#         fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.ecc, name=roi, line_color = roi_colors[j], meanline_visible=True))

#         # Set axis titles only for the left-most column and bottom-most row
#         fig.update_yaxes(title_text='pRF eccentricity (dva)')
#         fig.update_xaxes(title_text='Visual areas')
#         #Add the name of the plot with the subject name
#         fig.add_annotation(x=0.25, y=15, showarrow=False, text=('{}'.format(subject)), font=dict(size=20))

#     fig.update_layout(height=fig_height, width=fig_width, template='simple_white')
#     fig.write_image('/home/mchevillard/disks/meso_S/{}_ecc_roi.pdf'.format(subject))

## Make subplot

In [82]:
rows, cols = 2,2
fig_height, fig_width = 1080,1920

fig_subplot_1 = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1, shared_xaxes=False)
fig_subplot_2 = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1, shared_yaxes=False)

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        fig=fig_subplot_1 if i==0 else fig_subplot_2

        df = data.loc[(data.subject == subject) & (data.roi == roi)]


        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.r2, name=roi, showlegend=True, points=False, scalemode='width', width=0.75, line_color = roi_colors[j], meanline_visible=True), row=1, col=1)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.sd, name=roi, showlegend=False, points=False, scalemode='width', width=0.75, line_color = roi_colors[j], meanline_visible=True), row=1, col=2)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.ecc, name=roi, showlegend=False, points=False, scalemode='width', width=0.75, line_color = roi_colors[j], meanline_visible=True), row=2, col=1)


        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='r2', row=1, col=1)
        fig.update_yaxes(title_text='pRF size(dva)', row=1, col=2)
        fig.update_yaxes(title_text='pRF eccentricity (dva)', row=2, col=1)
        fig.update_xaxes(title_text='Visual areas', row=2, col=col)

    fig.update_layout(title= 'pRF parameters VS Visual areas {}'.format(subject), height=fig_height, width=fig_width,showlegend=True, template='simple_white', font=dict(size=16))

    fig.write_image('/home/mchevillard/disks/meso_S/{}_subplot_pRF.pdf'.format(subject))

## Make subplot shared

In [14]:
rows, cols = 2,2
fig_height, fig_width = 1080,1920

fig_subplot_loo_1 = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1)
fig_subplot_loo_2 = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1)

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        fig=fig_subplot_loo_1 if i==0 else fig_subplot_loo_2

        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        df_loo = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi == roi)]
        
        df = df.sort_values('r2', ascending=False)
        df_loo = df_loo.sort_values('r2', ascending=False)
        df = df.head(250)
        df_loo = df_loo.head(250)

        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.r2, name=roi, showlegend=True, legendgroup='avg', points=False, scalemode='width', width=0.75, side='negative', line_color = roi_colors[j], meanline_visible=True), row=1, col=1)
        fig.add_trace(go.Violin(x=df_loo['roi'][df_loo['roi']==roi], y=df_loo.r2, name=roi, showlegend=True, legendgroup='loo', points=False, scalemode='width', width=0.75, side='positive', line_color = roi_colors_loo[j], meanline_visible=True, fillcolor='rgb(255,255,255)'), row=1, col=1)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.sd, name=roi, showlegend=False, legendgroup='avg', points=False, scalemode='width', width=0.75, side='negative', line_color = roi_colors[j], meanline_visible=True), row=1, col=2)
        fig.add_trace(go.Violin(x=df_loo['roi'][df_loo['roi']==roi], y=df_loo.sd, name=roi, showlegend=False, legendgroup='loo', points=False, scalemode='width', width=0.75, side='positive', line_color=roi_colors_loo[j], meanline_visible=True, fillcolor='rgb(255,255,255)'), row=1, col=2)
        fig.add_trace(go.Violin(x=df['roi'][df['roi']==roi], y=df.ecc, name=roi, showlegend=False, legendgroup='avg', points=False, scalemode='width', width=0.75, side='negative', line_color = roi_colors[j], meanline_visible=True), row=2, col=1)
        fig.add_trace(go.Violin(x=df_loo['roi'][df_loo['roi']==roi], y=df_loo.ecc, name=roi, showlegend=False, legendgroup='loo', points=False, scalemode='width', width=0.75, side='positive', line_color = roi_colors_loo[j], meanline_visible=True, fillcolor='rgb(255,255,255)'), row=2, col=1)
        
        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(range=[0,1], title_text='r2', row=1, col=1)
        fig.update_yaxes(range=[0,20], title_text='pRF size(dva)', row=1, col=2)
        fig.update_yaxes(range=[0,15], nticks=4, title_text='pRF eccentricity (dva)', row=2, col=1)
        fig.update_xaxes(showline=True, ticklen=0, linecolor=('rgba(255,255,255,0)'), tickfont=dict(size=18))
        fig.update_traces(spanmode='manual', span=[0,1], row=1, col=1)  
        fig.update_traces(spanmode='manual', span=[0.1,20], row=1, col=2)
        fig.update_traces(spanmode='manual', span=[0,15], row=2, col=1)
    fig.update_layout(height=fig_height, width=fig_width, showlegend=True, legend=dict(orientation="h", yanchor='top', y=1.15, xanchor='left', x=0.22, traceorder='normal', itemwidth=50), template='simple_white', font=dict(size=16))
  
    fig.write_image('/home/mchevillard/disks/meso_S/figures/{}_subplot_pRF_loo.pdf'.format(subject))

In [14]:
# legend=dict(orientation="h", yanchor='top', y=1.1, xanchor="center", x=0.5, traceorder='grouped')

#

## Older version

In [7]:
#subplot settings SIZE
rows, cols = 1,2
fig_height, fig_width = 1080,1920

fig = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1)

roi_colors = px.colors.sequential.Sunset[:] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        if i ==1 :
            showlegend = True
        else :
            showlegend = False
        col = (i % 2) + 1
        row = i // 2 +1
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        slope, intercept = np.polyfit(df.ecc,df.sd,deg=1, w=df.r2)
        line = slope * df.ecc + intercept
        
        fig.add_trace(go.Scatter(x=df.ecc, y=line, mode='lines',name=roi, legendgroup=roi,line_color = roi_colors[j],showlegend=showlegend),row=row, col=col)
        
        
        # Set axis titles only for the left-most column and bottom-most row
        if col == 1:
            fig.update_yaxes(title_text='pRF size (dva)', row=row, col=col)
        if row == 1:
            fig.update_xaxes(title_text='pRF eccentricity (dva)', row=row, col=col)
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0.25, y=16, showarrow=False, text=('{}'.format(subject)), row=1, col=i + 1, font=dict(size=20))
    
fig.update_layout(height=fig_height, width=fig_width,showlegend=True, template='simple_white')

fig.write_image('/home/mchevillard/disks/meso_S/size_ecckjhj.pdf')